In [1]:
partition = 300

In [2]:
import sys
from train import main
from itertools import product  
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

n_tree_values = [5, 10, 15, 20]
tree_depth_values = [3, 5, 10]
batch_size_values = [256, 512, 1000]

best_score = 0
best_config = {}

for n_tree, tree_depth, batch_size in product(n_tree_values, tree_depth_values, batch_size_values):
    print(f"\nRunning: n_tree={n_tree}, tree_depth={tree_depth}, batch_size={batch_size}")
    sys.argv = [
        'train.py',
        '-dataset', f'gtd{partition}',
        '-n_class', '30',
        '-gpuid', '0',
        '-n_tree', str(n_tree),
        '-tree_depth', str(tree_depth),
        '-batch_size', str(batch_size),
        '-epochs', '200',
        '-jointly_training'
    ]
    
    _, _, _ = main()

    # Read best score from file (assumes one run per file)
    result_file = f"results/result_gtd{partition}"
    with open(result_file, "r") as f:
        lines = f.readlines()
        for line in lines:
            if "Best Accuracy" in line:
                acc = float(line.split()[2])
                if acc > best_score:
                    best_score = acc
                    best_config = {
                        'n_tree': n_tree,
                        'tree_depth': tree_depth,
                        'batch_size': batch_size
                    }
print("\nBest hyperparameter configuration:")
print(best_config)
print(f"Best accuracy: {best_score}")



Running: n_tree=5, tree_depth=3, batch_size=256
Use gtd300 dataset
Train Epoch: 1 [0/6300 (0%)]	Loss: 3.399434
Train Epoch: 1 [2560/6300 (40%)]	Loss: 3.366494
Train Epoch: 1 [5120/6300 (80%)]	Loss: 3.337430

Test set: Average loss: 3.3160, Accuracy: 595/2700 (0.2204)

Train Epoch: 2 [0/6300 (0%)]	Loss: 3.314532
Train Epoch: 2 [2560/6300 (40%)]	Loss: 3.288235
Train Epoch: 2 [5120/6300 (80%)]	Loss: 3.262682

Test set: Average loss: 3.2545, Accuracy: 760/2700 (0.2815)

Train Epoch: 3 [0/6300 (0%)]	Loss: 3.263672
Train Epoch: 3 [2560/6300 (40%)]	Loss: 3.229420
Train Epoch: 3 [5120/6300 (80%)]	Loss: 3.222390

Test set: Average loss: 3.2069, Accuracy: 820/2700 (0.3037)

Train Epoch: 4 [0/6300 (0%)]	Loss: 3.201795
Train Epoch: 4 [2560/6300 (40%)]	Loss: 3.192713
Train Epoch: 4 [5120/6300 (80%)]	Loss: 3.177704

Test set: Average loss: 3.1649, Accuracy: 924/2700 (0.3422)

Train Epoch: 5 [0/6300 (0%)]	Loss: 3.157274
Train Epoch: 5 [2560/6300 (40%)]	Loss: 3.152238
Train Epoch: 5 [5120/6300 (80%)]

In [3]:
sys.argv = [
    'train.py',
    '-dataset', f'gtd{partition}',
    '-n_class', '30',
    '-gpuid', '0',
    '-n_tree', str(best_config['n_tree']),
    '-tree_depth', str(best_config['tree_depth']),
    '-batch_size', str(best_config['batch_size']),
    '-epochs', '200',
    '-jointly_training'
]

preds, targets, labels = main()


Use gtd300 dataset
Train Epoch: 1 [0/6300 (0%)]	Loss: 3.401179
Train Epoch: 1 [5120/6300 (77%)]	Loss: 3.341071

Test set: Average loss: 3.3219, Accuracy: 915/2700 (0.3389)

Train Epoch: 2 [0/6300 (0%)]	Loss: 3.318545
Train Epoch: 2 [5120/6300 (77%)]	Loss: 3.251294

Test set: Average loss: 3.2341, Accuracy: 1026/2700 (0.3800)

Train Epoch: 3 [0/6300 (0%)]	Loss: 3.231806
Train Epoch: 3 [5120/6300 (77%)]	Loss: 3.164822

Test set: Average loss: 3.1579, Accuracy: 1133/2700 (0.4196)

Train Epoch: 4 [0/6300 (0%)]	Loss: 3.147175
Train Epoch: 4 [5120/6300 (77%)]	Loss: 3.100584

Test set: Average loss: 3.0983, Accuracy: 1253/2700 (0.4641)

Train Epoch: 5 [0/6300 (0%)]	Loss: 3.088819
Train Epoch: 5 [5120/6300 (77%)]	Loss: 3.051688

Test set: Average loss: 3.0486, Accuracy: 1342/2700 (0.4970)

Train Epoch: 6 [0/6300 (0%)]	Loss: 3.043623
Train Epoch: 6 [5120/6300 (77%)]	Loss: 3.007528

Test set: Average loss: 3.0050, Accuracy: 1492/2700 (0.5526)

Train Epoch: 7 [0/6300 (0%)]	Loss: 3.002420
Train Ep

In [4]:
def plot_confusion_matrix(y_true, y_pred, labels, partition):
    cm = confusion_matrix(y_true, y_pred, labels=range(len(labels)))
    cm_normalized = cm.astype('float') / cm.sum(axis=1, keepdims=True)

    plt.figure(figsize=(18, 16))
    sns.heatmap(cm_normalized,
                annot=True,
                fmt=".2f",
                xticklabels=labels,
                yticklabels=labels,
                cmap="viridis",
                square=True,
                linewidths=0.5,
                cbar_kws={"shrink": 0.8})

    plt.title(f"Normalized Confusion Matrix (Partition gtd{partition})", fontsize=18)
    plt.xlabel("Predicted Label", fontsize=14)
    plt.ylabel("True Label", fontsize=14)
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()

    save_path = f"results/confusion_matrix_partition_gtd{partition}.png"
    plt.savefig(save_path, dpi=300)
    plt.close()

    print(f"Saved confusion matrix for partition gtd{partition} to {save_path}")



In [5]:
plot_confusion_matrix(targets, preds, labels, partition)

Saved confusion matrix for partition gtd300 to results/confusion_matrix_partition_gtd300.png
